<a href="https://colab.research.google.com/github/Flaviocs/RAG/blob/main/RAG_excel_gpt_qdrant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-cloud-aiplatform pymysql faiss-cpu

Autenticação com o Google Cloud

In [13]:
import pandas as pd

df = pd.read_excel("/content/drive/MyDrive/Portfolio_ninja.xlsx", sheet_name=0)
df = df.drop(columns=['Unnamed: 7'])
df.head()

,idProduto,DescrTituloProduto,DescrProduto,DescrConteudoProgramatico,NrCargaHoraria,pago,VlrPrecificacao
0,1,Orientação Teste,Teste,TESTE. Finalidade TESTEas dsad,60,0.0,0.0
1,3,Na Medida - Como criar um site de sucesso,"Neste ""Na Medida"", o participante conhece as p...",1) introdução 2) internet: oportunidades para ...,180,1.0,70.0
2,14,Na Medida - Gestão de Pessoas e Equipes,O curso conta com atividades práticas que irão...,1) Introdução 2) Gestão de pessoas 3) Sistemas...,1440,0.0,0.0
3,15,Na Medida - Gestão Estratégica de Vendas,O curso auxilia o empresário a estruturar seu ...,1) Introdução 2) Analisando o processo de comp...,960,0.0,0.0
4,16,Na Medida - Gestão Financeira,"O curso ensina gestão financeira, passando por...",- Planejamento financeiro - Fluxo de...,1200,0.0,0.0


Geração dos embeddings (ex: com OpenAI)

In [15]:
from openai import OpenAI
import openai

openai.api_key = ""

def gerar_embedding(texto):
  # Converte a entrada para string e lida com potenciais erros
  try:
    texto = str(texto)  # Garante que a entrada seja uma string
    response = openai.embeddings.create(input=texto, model="text-embedding-ada-002")
    return response.data[0].embedding
  except Exception as e:
    print(f"Erro ao processar a entrada: {texto}, Erro: {e}")
    return None  # Ou lida com o erro de uma forma que atenda às suas necessidades

df["embedding"] = df["DescrProduto"].apply(gerar_embedding)

In [16]:
df.head()

,idProduto,DescrTituloProduto,DescrProduto,DescrConteudoProgramatico,NrCargaHoraria,pago,VlrPrecificacao,embedding
0,1,Orientação Teste,Teste,TESTE. Finalidade TESTEas dsad,60,0.0,0.0,"[-0.021639620885252953, 0.005038257222622633, ..."
1,3,Na Medida - Como criar um site de sucesso,"Neste ""Na Medida"", o participante conhece as p...",1) introdução 2) internet: oportunidades para ...,180,1.0,70.0,"[0.005513374228030443, 0.00820610485970974, 0...."
2,14,Na Medida - Gestão de Pessoas e Equipes,O curso conta com atividades práticas que irão...,1) Introdução 2) Gestão de pessoas 3) Sistemas...,1440,0.0,0.0,"[-0.02393907494843006, -0.02223820798099041, 0..."
3,15,Na Medida - Gestão Estratégica de Vendas,O curso auxilia o empresário a estruturar seu ...,1) Introdução 2) Analisando o processo de comp...,960,0.0,0.0,"[-0.03199392557144165, -0.016783909872174263, ..."
4,16,Na Medida - Gestão Financeira,"O curso ensina gestão financeira, passando por...",- Planejamento financeiro - Fluxo de...,1200,0.0,0.0,"[-0.02753019519150257, -0.017018666490912437, ..."


Persistencia dos Embeddings

In [17]:
!pip install qdrant-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.7/327.7 kB 7.4 MB/s eta 0:00:00


In [22]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams

# Substitua com suas credenciais
URL_QDRANT = ""
API_KEY = ""

# Inicializa o cliente
client = QdrantClient(url=URL_QDRANT, api_key=API_KEY)

if not client.collection_exists("produtos"):
    client.create_collection(
        collection_name="produtos",
        vectors_config=VectorParams(size=1536, distance=Distance.COSINE)
    )

# 2. Prepara os pontos
pontos = [
    PointStruct(
        id=int(i),
        vector=row["embedding"],
        payload={
            "titulo": row["DescrTituloProduto"],
            "descricao": row["DescrProduto"],
            "idProduto":row["idProduto"]
        }
    )
    for i, row in df.iterrows()
]

# 3. Envia os pontos em lotes menores
def enviar_em_lotes(client, collection_name, pontos, batch_size=100):
    for i in range(0, len(pontos), batch_size):
        batch = pontos[i:i+batch_size]
        client.upsert(collection_name=collection_name, points=batch)
        print(f"Lote {i} a {i+len(batch)} enviado com sucesso.")

enviar_em_lotes(client, "produtos", pontos, batch_size=100)


Lote 0 a 100 enviado com sucesso.
Lote 100 a 200 enviado com sucesso.
Lote 200 a 300 enviado com sucesso.
Lote 300 a 400 enviado com sucesso.
Lote 400 a 500 enviado com sucesso.
Lote 500 a 600 enviado com sucesso.
Lote 600 a 700 enviado com sucesso.
Lote 700 a 800 enviado com sucesso.
Lote 800 a 900 enviado com sucesso.
Lote 900 a 1000 enviado com sucesso.
Lote 1000 a 1100 enviado com sucesso.
Lote 1100 a 1200 enviado com sucesso.
Lote 1200 a 1300 enviado com sucesso.
Lote 1300 a 1400 enviado com sucesso.
Lote 1400 a 1500 enviado com sucesso.
Lote 1500 a 1600 enviado com sucesso.
Lote 1600 a 1700 enviado com sucesso.
Lote 1700 a 1800 enviado com sucesso.
Lote 1800 a 1900 enviado com sucesso.
Lote 1900 a 1923 enviado com sucesso.


busca semântica depois

In [28]:
# Gera embedding da pergunta
pergunta = "ajuda sobre agentes automatizados?"
embedding_pergunta = gerar_embedding(pergunta)

# Consulta os vetores mais similares
resultados = client.search(
    collection_name="produtos",
    query_vector=embedding_pergunta,
    limit=10
)

# Exibe os melhores resultados
for r in resultados:
    print(f"ID: {r.id}")
    print(f"idProduto: {r.payload['idProduto']}")
    print(f"Título: {r.payload['titulo']}")
    print(f"Descrição: {r.payload['descricao']}")
    print("------")


<ipython-input-28-5d656d5ccf50>:6: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  resultados = client.search(


ID: 1605
idProduto: 4724
Título: Sebraetec Senai - Elaboração em implementação de projeto de automação industrial (80 hs) (Presencial)
Descrição: Consultoria tecnológica para implantação da Automação Industrial.
------
ID: 1603
idProduto: 4722
Título: Sebraetec Senai - Elaboração em implementação de projeto de automação industrial (60 hs) (Presencial)
Descrição: Consultoria tecnológica para implantação da Automação Industrial.
------
ID: 1285
idProduto: 4032
Título: Suporte Sebrae Aqui 
Descrição: Apoio ao agente dos postos para atendimentos realizados pela equipe SEBRAE AQUI  
------
ID: 1392
idProduto: 4297
Título: Acionamento de motores para Automação Predial (Presencial)
Descrição: Curso do SEBRAE e do SENAI que ajuda a desenvolver competências necessárias sobre técnicas para Acionamento de motores para Automação Predial, de acordo com procedimentos e normas técnicas, ambientais, de qualidade, de saúde e segurança no trabalho.
------
ID: 827
idProduto: 3262
Título: Plano de comerci

Fazendo o RAG com o GPT

In [30]:
# Transformar a pergunta em embedding
pergunta = "Quais produtos ajudam com redes sociais?"
embedding_pergunta = gerar_embedding(pergunta)

# Buscar no Qdrant
resultado = client.search(
    collection_name="produtos",
    query_vector=embedding_pergunta,
    limit=3
)

# Montar contexto para GPT
contexto = "\n".join([f"{r.payload['titulo']}: {r.payload['descricao']}" for r in resultado])

# Enviar para GPT
resposta = openai.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "Você é um assistente que responde com base nos produtos listados."},
        {"role": "user", "content": f"Contexto:\n{contexto}\n\nPergunta: {pergunta}"}
    ]
)

print(resposta.choices[0].message.content)


<ipython-input-30-4411c0fe0446>:6: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  resultado = client.search(


Os produtos que ajudam com redes sociais são:

1. Serviço URC: OT Evento - Assíncrono Semana Digital: Este serviço proporciona orientação para auxiliar o empresário a aplicar técnicas de impulsionamento de redes sociais na empresa. 

2. Serviço URC: OT Evento Assíncrono - Setorial Comércio: Este é um serviço orientado para o setor de comércio que auxilia os empresários a colocar em prática todas as técnicas de impulsionamento de redes sociais em sua empresa.

3. Serviço URC: OT Evento Assíncrono - Setorial Agro: Este serviço é focado no setor agro e fornece orientação para auxiliar o empresário a implementar técnicas de impulsionamento de redes sociais na empresa.
